#1. Ajuste de NER para un modelo pre entrenado de BERT para el quechua (modelo preentrenado desde cero, sin extender)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NER-quechua-webapp/

/content/drive/MyDrive/NER-quechua-webapp


In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
data = pd.read_csv("input/ner_dataset.csv")

In [ ]:
data #No tiene POS asignado.

,Sentence #,Word,POS,Tag
0,Sentence: 1,Tiñiqi,NNS,B-ORG
1,NaN,(,NNS,O
2,NaN,'cilindro,NNS,O
3,NaN,',NNS,O
4,NaN,),NNS,O
...,...,...,...,...
13931,NaN,López,NNS,I-PER
13932,NaN,de,NNS,I-PER
13933,NaN,Romaña,NNS,I-PER
13934,NaN,',NNS,O


In [ ]:
data = data.fillna(method = "ffill")

In [ ]:
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Tiñiqi,NNS,B-ORG
1,Sentence: 1,(,NNS,O
2,Sentence: 1,'cilindro,NNS,O
3,Sentence: 1,',NNS,O
4,Sentence: 1,),NNS,O
...,...,...,...,...
13931,Sentence: 2454,López,NNS,I-PER
13932,Sentence: 2454,de,NNS,I-PER
13933,Sentence: 2454,Romaña,NNS,I-PER
13934,Sentence: 2454,',NNS,O


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [ ]:
data.head(30)

,Sentence #,Word,POS,Tag
0,0,Tiñiqi,NNS,B-ORG
1,0,(,NNS,O
2,0,'cilindro,NNS,O
3,0,',NNS,O
4,0,),NNS,O
5,1111,Lluq'i,NNS,B-LOC
6,1111,',NNS,O
7,1111,(,NNS,O
8,1111,Kageneckia,NNS,O
9,1111,lanceolata,NNS,O


In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
train_data

,sentence_id,words,labels
4565,2235,Kunth,I-PER
3982,2123,(,I-LOC
2264,1780,parki,I-ORG
4068,2138,Santa,B-LOC
8374,519,hayk'appas,O
...,...,...,...
4576,2237,Liyun,B-PER
6979,252,',O
436,2188,Qulan,B-ORG
482,2299,***,O


In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:
label = data["labels"].unique().tolist()
label

['B-ORG', 'O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-ORG']

In [ ]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

#2. Se ha creado el NERModel a partir de los archivos generados anteriormente. Un BERT Model pre entrenado para quechua desde cero (sin extender).

In [ ]:
model = NERModel('bert', "input/bert-base-uncased" ,labels=label,args =args,use_cuda=True)

Some weights of the model checkpoint at input/bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized fr

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:775: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/77 [00:00<?, ?it/s]

(770, 0.6861677180637012)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/54 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 1.8665361271964178,
 'f1_score': 0.24737678855325912,
 'precision': 0.22695449241540255,
 'recall': 0.2718378756114605}

#3. Algunos resultados con el nuevo modelo NER para el BERT sin extender, hecho desde cero.

In [ ]:
prediction, model_output = model.predict(["Sam"]) #Sam ->Un nombre.
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Sam': 'I-ORG'}]]

In [ ]:
prediction, model_output = model.predict(["Allinllachu? Estados Unidosmantam kani, Sutiymi, Sam."]) #¿Cómo estás? Mi nombre es Sam y soy de Estados Unidos
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Allinllachu?': 'O'},
  {'Estados': 'O'},
  {'Unidosmantam': 'O'},
  {'kani,': 'O'},
  {'Sutiymi,': 'O'},
  {'Sam.': 'O'}]]

In [ ]:
prediction, model_output = model.predict(["Uma llaqtanqa Morelia llaqtam."]) #B-LOC, I-LOC,B-LOC, O, justo es lo mismo del dataset con el que se entrenó.
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Uma': 'B-LOC'},
  {'llaqtanqa': 'I-LOC'},
  {'Morelia': 'B-LOC'},
  {'llaqtam.': 'O'}]]

In [ ]:
prediction, model_output = model.predict(["Imaynataq riyman Antabamba llaqtaman?"]) #¿Cómo llego a Antabamba? 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Imaynataq': 'B-ORG'},
  {'riyman': 'I-ORG'},
  {'Antabamba': 'B-ORG'},
  {'llaqtaman?': 'I-ORG'}]]

# 4. Desplegar Aplicativo Web

**Desplegar el sitio web temporal**

In [ ]:
!pip install -q pyngrok
from pyngrok import ngrok

In [ ]:
url = ngrok.connect(8501)
url.public_url

'http://1f060e755835.ngrok.io'

**Ejecutar como webapp**

In [ ]:
!simple-viewer


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.141.68:8501

2021-07-24 13:52:32.211557: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-24 13:52:37.635  Converting to features started.
100% 1/1 [00:00<00:00, 11008.67it/s]
Running Prediction: 100% 1/1 [00:00<00:00,  1.73it/s]
2021-07-24 13:52:50.643  Converting to features started.
100% 1/1 [00:00<00:00, 13706.88it/s]
Running Prediction: 100% 1/1 [00:00<00:00,  1.85it/s]
2021-07-24 14:22:12.078  Converting to features started.
100% 1/1 [00:00<00:00, 2211.02it/s]
Running Prediction: 100% 1/1 [00:00<00:00,  1.87it/s]
2021-07-24 14:22:17.001  Converting to features started.
100% 1/1 [00:00<00:00, 2366.99it/s]
Running Prediction: 100% 1/1 [00:00<00:00,  1.88it/s]
  Stopping...
